In [18]:
import pandas as pd
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import os
from torchvision import transforms

In [19]:
class dc_dataset(Dataset):
    def __init__(self, path ,transform=None):
        self.path = path
        self.img_list = os.listdir(path)
        self.transform = transform

    def __getitem__(self, idx):
        if self.img_list[idx][0] =='d':
            label = 0
        else:
            label = 1
        image = Image.open( os.path.join(self.path, self.img_list[idx]) ).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, label
    
    def __len__(self):
        return len(self.img_list)

In [20]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample

    def forward(self, x):
        identity = x
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        if self.downsample is not None:
            identity = self.downsample(x)
        out += identity
        out = self.relu(out)
        return out

def make_layer(block, in_channels, out_channels, num_blocks, stride):
    downsample = None
    if stride != 1 or in_channels != out_channels:
        downsample = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
            nn.BatchNorm2d(out_channels)
        )
    layers = []
    layers.append(block(in_channels, out_channels, stride, downsample))
    for _ in range(1, num_blocks):
        layers.append(block(out_channels, out_channels))
    return nn.Sequential(*layers)

class ResNet20(nn.Module):
    def __init__(self, num_classes=2):
        super(ResNet20, self).__init__()
        self.in_channels = 16
        self.conv = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)
        # 각 스테이지마다 3개 블록
        self.layer1 = make_layer(ResidualBlock, 16, 16, 3, stride=1)
        self.layer2 = make_layer(ResidualBlock, 16, 32, 3, stride=2)
        self.layer3 = make_layer(ResidualBlock, 32, 64, 3, stride=2)
        self.avg_pool = nn.AvgPool2d(8)
        self.fc = nn.Linear(64, num_classes)

    def forward(self, x):
        out = self.relu(self.bn(self.conv(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

# 사용 예시
# model = ResNet20(num_classes=2)


In [21]:
model = ResNet20(num_classes=2)

In [22]:
transform = transforms.Compose([
    transforms.Resize((32,32)),
    transforms.RandomCrop(32, padding=4),      # 패딩 후 랜덤 크롭
    transforms.RandomHorizontalFlip(),         # 랜덤 좌우반전
    transforms.ToTensor(),                     # 텐서 변환
])


# 1.2 커스텀 데이터셋 클래스에 트랜스폼 적용
train_dataset = dc_dataset('./train', transform=transform)
test_dataset = dc_dataset('./test', transform=transform)

# 1.3 DataLoader로 배치 단위 데이터 준비
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=2)

In [23]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ResNet20(num_classes=2).to(device)  # 클래스 수 맞게 수정

In [24]:
import torch
import torch.nn as nn
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=1e-4)  # 학습률 0.01로 낮춤
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[15, 30], gamma=0.1)  # milestone도 더 이르게

In [25]:
def train_one_epoch(model, loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * images.size(0)
    return running_loss / len(loader.dataset)

In [26]:
def evaluate(model, loader, criterion, device):
    model.eval()
    correct = 0
    total = 0
    test_loss = 0.0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            test_loss += criterion(outputs, labels).item() * images.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    avg_loss = test_loss / total
    return avg_loss, accuracy

In [27]:
num_epochs = 40  # 논문 기준
for epoch in range(1, num_epochs + 1):
    train_loss = train_one_epoch(model, train_loader, criterion, optimizer, device)
    test_loss, test_acc = evaluate(model, test_loader, criterion, device)
    scheduler.step()
    print(f"Epoch {epoch:3d} | Train Loss: {train_loss:.4f} | Test Loss: {test_loss:.4f} | Test Acc: {test_acc:.2f}%")

Epoch   1 | Train Loss: 0.6258 | Test Loss: 0.7973 | Test Acc: 56.45%
Epoch   2 | Train Loss: 0.5370 | Test Loss: 0.5596 | Test Acc: 73.85%
Epoch   3 | Train Loss: 0.4808 | Test Loss: 0.5049 | Test Acc: 75.28%
Epoch   4 | Train Loss: 0.4388 | Test Loss: 0.4070 | Test Acc: 81.91%
Epoch   5 | Train Loss: 0.4074 | Test Loss: 0.3634 | Test Acc: 84.58%
Epoch   6 | Train Loss: 0.3728 | Test Loss: 0.3670 | Test Acc: 83.93%
Epoch   7 | Train Loss: 0.3413 | Test Loss: 0.3555 | Test Acc: 84.18%
Epoch   8 | Train Loss: 0.3282 | Test Loss: 0.3325 | Test Acc: 86.16%
Epoch   9 | Train Loss: 0.3105 | Test Loss: 0.2876 | Test Acc: 87.89%
Epoch  10 | Train Loss: 0.2974 | Test Loss: 0.2830 | Test Acc: 88.38%
Epoch  11 | Train Loss: 0.2840 | Test Loss: 0.7242 | Test Acc: 71.82%
Epoch  12 | Train Loss: 0.2715 | Test Loss: 0.2452 | Test Acc: 89.87%
Epoch  13 | Train Loss: 0.2602 | Test Loss: 0.2450 | Test Acc: 89.32%
Epoch  14 | Train Loss: 0.2549 | Test Loss: 0.2474 | Test Acc: 89.72%
Epoch  15 | Train Lo

In [29]:
torch.save(model.state_dict(), 'resnet20.pth')